In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('FF_data')
print("connected to sqlite3 verion:", sqlite3.version)

connected to sqlite3 verion: 2.6.0


In [3]:
lastChronicle = pd.read_sql_query(
     """SELECT *
              
       from fracChronicle"""
     , conn)

In [4]:
lastChronicle.tail()

,APINumber,OperatorName,JobEndDate,pullDate
173914,37015217420000,"EOG Resources, Inc.",10/14/2020 12:00:00 AM,2020-10-18
173915,37015219880000,"EOG Resources, Inc.",10/14/2020 12:00:00 AM,2020-10-18
173916,42173346750000,Berry Petroleum,5/23/2021 12:00:00 AM,2020-10-18
173917,42115334560000,Pioneer Natural Resources,6/8/2021 12:00:00 AM,2020-10-18
173918,42227368950000,"Meritage Energy Company, LLC",4/1/3012 12:00:00 AM,2020-10-18


In [5]:
lastUpdate = pd.read_sql_query(
     """SELECT APINumber
              ,OperatorName
              ,JobEndDate
              
        from RegistryUpload"""
       ,conn)

In [6]:
lastUpdate['pullDate'] = pd.to_datetime('today').date()
lastUpdate = lastUpdate.drop_duplicates().reset_index(drop=True)
lastUpdate.tail()

,APINumber,OperatorName,JobEndDate,pullDate
174023,05071084630000,Evergreen Natural Resources LLC,10/21/2020 6:00:00 AM,2020-10-25
174024,42283370210000,"Chesapeake Operating, Inc.",10/22/2020 1:51:56 AM,2020-10-25
174025,42173346750000,Berry Petroleum,5/23/2021 12:00:00 AM,2020-10-25
174026,42115334560000,Pioneer Natural Resources,6/8/2021 12:00:00 AM,2020-10-25
174027,42227368950000,"Meritage Energy Company, LLC",4/1/3012 12:00:00 AM,2020-10-25


In [7]:
df_all = lastChronicle.merge(lastUpdate.drop_duplicates(), on=['APINumber','JobEndDate'], 
                   how='outer', indicator=True)

In [8]:
df_all = df_all.drop_duplicates().reset_index(drop=True)
df_all.head()

,APINumber,OperatorName_x,JobEndDate,pullDate_x,OperatorName_y,pullDate_y,_merge
0,49009219470000,"Chesapeake Operating, Inc.",5/19/1982 12:00:00 AM,2020-10-18,"Chesapeake Operating, Inc.",2020-10-25,both
1,49009228850000,"Chesapeake Operating, Inc.",2/7/1995 12:00:00 AM,2020-10-18,"Chesapeake Operating, Inc.",2020-10-25,both
2,42335355480000,Energen Resources Corporation,6/11/1996 12:00:00 AM,2020-10-18,Energen Resources Corporation,2020-10-25,both
3,42395313840000,XTO Energy/ExxonMobil,12/13/2001 12:00:00 AM,2020-10-18,XTO Energy/ExxonMobil,2020-10-25,both
4,37059247370000,CONSOL Energy Inc.,9/22/2008 12:00:00 AM,2020-10-18,CONSOL Energy Inc.,2020-10-25,both


In [9]:
fracChronicle = df_all[['APINumber','JobEndDate']].copy()
fracChronicle['OperatorName'] = df_all['OperatorName_x'].fillna(df_all['OperatorName_y'])
fracChronicle['pullDate'] = df_all['pullDate_x'].fillna(df_all['pullDate_y'])
fracChronicle['pullDate'] = pd.to_datetime(fracChronicle['pullDate'])
fracChronicle = fracChronicle.drop_duplicates().reset_index(drop=True)

In [10]:
fracChronicle.tail(2)

,APINumber,JobEndDate,OperatorName,pullDate
174028,05071084630000,10/21/2020 6:00:00 AM,Evergreen Natural Resources LLC,2020-10-25
174029,42283370210000,10/22/2020 1:51:56 AM,"Chesapeake Operating, Inc.",2020-10-25


In [11]:
fracsDropped = df_all[(df_all['_merge'] == 'left_only')]
fracsDropped

,APINumber,OperatorName_x,JobEndDate,pullDate_x,OperatorName_y,pullDate_y,_merge
141913,05123426480000,Whiting Petroleum,4/30/2018 4:55:05 PM,2020-10-18,NaN,NaN,left_only
143188,05123457080000,HighPoint Operating Corporation,5/25/2018 6:00:00 AM,2020-10-18,NaN,NaN,left_only


In [12]:
fracsAdded = fracChronicle[fracChronicle['pullDate'] == '2020-10-25']
fracsAdded

,APINumber,JobEndDate,OperatorName,pullDate
173919,35071248800000,1/29/2018 6:00:00 AM,Blue Ox Partners LLC,2020-10-25
173920,05123426480000,4/30/2018 6:00:00 AM,Whiting Petroleum,2020-10-25
173921,05123461340000,2/28/2019 7:00:00 AM,Crestone Peak Resources,2020-10-25
173922,42341005830000,5/15/2019 12:00:00 AM,Scout Energy Partners,2020-10-25
173923,42421006130000,5/16/2019 12:00:00 AM,Scout Energy Partners,2020-10-25
...,...,...,...,...
174025,30015466000000,10/16/2020 5:00:00 AM,COG Operating LLC,2020-10-25
174026,33053081160000,10/16/2020 5:00:00 AM,"Petro Hunt, L.L.C.",2020-10-25
174027,05071092650000,10/19/2020 6:00:00 AM,Evergreen Natural Resources LLC,2020-10-25
174028,05071084630000,10/21/2020 6:00:00 AM,Evergreen Natural Resources LLC,2020-10-25


In [13]:
fracChronicle.to_sql('fracChronicle', conn, index=False, if_exists='replace')

In [14]:
conn.close()